In [13]:
import cv2
import numpy as np
import os
import argparse
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, losses
from random import shuffle

In [14]:

class_name1 = "person1"
class_name2 = "person2"
n_samples = 40

# Create directories for the classes
os.makedirs(class_name1, exist_ok=True)
os.makedirs(class_name2, exist_ok=True)

def generate_dataset(class_name):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    img_id = 0

    while img_id < n_samples:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(f"{class_name}/img{img_id}.jpg", face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples for {class_name} is completed !!!")

generate_dataset(class_name1)
generate_dataset(class_name2)

def load_data(path):
    data = []
    for imgs in os.listdir(path):
        img = np.array(Image.open(f'{path}/{imgs}'))
        data.append(img)
    return data

def process_data(c1, c2):
    data = []
    x = []
    y = []
    for img in c1:
        data.append([img, [1, 0]])
    for img in c2:
        data.append([img, [0, 1]])
    for d in data:
        x.append(d[0])
    for d in data:
        y.append(d[1])
    return np.array(x), np.array(y)

def get_model(n_class):
    model = tf.keras.Sequential([
        tf.keras.Input((200, 200, 1)),
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(n_class, activation='softmax'),
    ])
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

class1, class2 = load_data(class_name1), load_data(class_name2)

x, y = process_data(class1, class2)

# Normalize data
x = x / 255.0
x = x.reshape(x.shape[0], 200, 200, 1)

model = get_model(n_class=2)

model.fit(x, y, epochs=20)

# Example prediction (needs an image path)
# print(model.predict(np.array(Image.open('path_to_image').resize((200, 200)).convert('L')).reshape(1, 200, 200, 1))[0])


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [10]:
print(model.predict(np.array(Image.open("X:\workspace\codespace\person2\img1.jpg").resize((200, 200)).convert('L')).reshape(1, 200, 200, 1))[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[0. 1.]
